# Reward Competition Elo rating Calculation

## Importing other Python Libraries/Modules

In [1]:
import copy
import re
import os
import sys
import string
import glob
import ast
from collections import Counter
from collections import defaultdict
import warnings

In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import git
# Getting the path of the root directory so that we can import repo specific functions
git_repo_object = git.Repo('.', search_parent_directories=True)
git_repo_directory = git_repo_object.working_tree_dir

In [4]:
# Setting path so that we can import functions
sys.path.append(os.path.join(git_repo_directory, "src"))

In [5]:
os.path.join(git_repo_directory, "src")


'/nancy/projects/social_competiton_elo_rating/src'

In [6]:
from elorating import calculation
from elorating import dataframe

In [7]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (18,10)

# Summary

- Experiment
    - Observing competitve behavior between two mice inside an operant chamber where tone is associated with a single liquid food reward being dispensed. Mice are prompted to compete for access to the reward.
- Data
    - Excel spreadsheet of recorded reward comeptition results. The relevant columns are those of the "Date" of the recording, the ID's of the two competing subjects, and the results for each trial. Each row will be all the trials for this pair for the recording session.
    - There are multiple cages for each sheet of the spreadsheet.
- Purpose of this Jupyter Notebook
    - To calculate the Elo rating of each mice after each interaction. The mice start off with an Elo rating of 1000. Elo ratings are calculated with the formula from here: https://www.omnicalculator.com/sports/elo . Then we will plot the change of Elo rating across all interactions. With the number of interactions on the X-Axis and the current Elo rating on the Y. There will be a line for mice.

## Name of protocol for naming

- This name will be used to name files and title plots. Please change if you are using a different protocol or adding more details
    - **NOTE**: This should be changed based on the name the protocol

In [8]:
protocol_name = "reward_competition"

In [9]:
prefix_name = "rc"

In [10]:
cohort_name = ""

In [11]:
if not cohort_name:
    cohort_name = input("""Type out the name of the cohort you are using. 
                        Make sure that this is typed as one word with individual words separated by underscore. i.e. pilot_3""")

Type out the name of the cohort you are using. 
                        Make sure that this is typed as one word with individual words separated by underscore. i.e. pilot_3


## Getting the file name of the raw data

- Default input folder and keyword to search the files for 
    - **NOTE**: This should not be changed unless there is a consistent change with the file naming convention

In [12]:
input_folder = os.path.join(".", "data")

In [13]:
# Accomodates for both capitalization of the file names
raw_data_file_keyword = '*[rR]eward*'

# NOTE: If there is a set excel file used for this notebook, make the cell below into a code cell and put the path in the "" (quotation marks)

In [14]:
raw_data_file_path = ""

- Asking the user what the path to the recording files are, with the option of using wildcards

In [15]:
if not raw_data_file_path:
    raw_data_glob_pattern = input("""Type out the path(address) of the raw behavioral recording excel sheets.
    Remember that if you are using a relative path, it will be based off of the location of this Jupyter Notebook.

    Globbing can also be used if you want to search with a wild card(Capitalization matters). 
    i.e. './data/*Home*' will be able to find './data/Homecage_observation.xlsx'

    NOTE: If left blank, the path will automatically be guessed off of the default settings 
    """)

Type out the path(address) of the raw behavioral recording excel sheets.
    Remember that if you are using a relative path, it will be based off of the location of this Jupyter Notebook.

    Globbing can also be used if you want to search with a wild card(Capitalization matters). 
    i.e. './data/*Home*' will be able to find './data/Homecage_observation.xlsx'

    NOTE: If left blank, the path will automatically be guessed off of the default settings 
    


- By default, this will search for files that are in the `./data` folder (in the folder that this notebook is in) that have key word you specified in the file name

In [16]:
if not raw_data_file_path:
    # Using the user inputted path/pattern
    if raw_data_glob_pattern.strip():
        # Getting a list of all matching files
        raw_data_files_list = glob.glob(raw_data_glob_pattern.strip(), recursive=True)
        # Checking if there were any files that matched
        if not raw_data_files_list:
            raise ValueError("No files were found with the path/pattern of {}. Please rerun the previous cell with the correct path".format(raw_data_glob_pattern))

    # Using the default pattern
    else:
        raw_data_glob_pattern = os.path.join(input_folder, raw_data_file_keyword)
        raw_data_files_list = glob.glob(raw_data_glob_pattern.strip(), recursive=True)
        # Searching for matching files with recursion
        if not raw_data_files_list:
            raise ValueError("No files were found in {} that had the keyword {} in it".format(input_folder, raw_data_file_keyword))


- Checking to see if only one file is specified

In [17]:
if not raw_data_file_path:
    if len(raw_data_files_list) >= 2:
        raise ValueError("More than one file was found with the path/pattern of {}. Please rerun the previous cell with the correct path".format(raw_data_glob_pattern))
    else:
        # Using the first(and only file) as the file path
        raw_data_file_path = raw_data_files_list[0]

In [18]:
raw_data_file_path

'./data/pilot_3_Reward_Comp_Scoring_Assignments.xlsx'

## Getting a list of all the sheets

In [19]:
# Getting the sheet names for the excel file
xls = pd.ExcelFile(raw_data_file_path)
raw_data_sheet_names = xls.sheet_names

In [20]:
raw_data_sheet_names

['Master List',
 'Jocelyn',
 'Aidan',
 'Albert',
 'Erika',
 'Ezra',
 'Meghan',
 'Naeliz',
 'Leo']

# NOTE: If there is a set sheet names used for this notebook, edit the cell below with the name of the sheets each in quotation marks seperated by commas in the brackets
   - i.e. `['CAGE 1', 'CAGE 2', 'CAGE 3', 'CAGE 4']`

In [21]:
inputted_sheet_names_list = []

- Asking the user what sheets they want to use

In [22]:
if not inputted_sheet_names_list:
    inputted_sheet_names_string = input("""Type out the name of the sheets that you want to be processed. 
    Each name must be put in quotes and seperated by a comma(,). i.e. "CAGE3", "CAGE4"

    The available sheets are: {}

    Alternatively, you can use the index of the list of names above. 
    Remember, that Python is zero indexed so the first item will be have the 0 index, second the 1 index, and so on. 
    i.e. 1, 2

    NOTE: If left blank, all sheets will be used
    """.format(raw_data_sheet_names))

Type out the name of the sheets that you want to be processed. 
    Each name must be put in quotes and seperated by a comma(,). i.e. "CAGE3", "CAGE4"

    The available sheets are: ['Master List', 'Jocelyn', 'Aidan', 'Albert', 'Erika', 'Ezra', 'Meghan', 'Naeliz', 'Leo']

    Alternatively, you can use the index of the list of names above. 
    Remember, that Python is zero indexed so the first item will be have the 0 index, second the 1 index, and so on. 
    i.e. 1, 2

    NOTE: If left blank, all sheets will be used
    


In [23]:
if not inputted_sheet_names_list:
    # Making a list out of the string of inputted sheet names
    if inputted_sheet_names_string:
        inputted_sheet_names_string = "[" + inputted_sheet_names_string + "]"
        # Turning the string into a list
        inputted_sheet_names_list = ast.literal_eval(inputted_sheet_names_string)
    # Using all the sheet names if no sheet name is specified
    else:
        inputted_sheet_names_list =  raw_data_sheet_names

In [24]:
inputted_sheet_names_list

['Master List',
 'Jocelyn',
 'Aidan',
 'Albert',
 'Erika',
 'Ezra',
 'Meghan',
 'Naeliz',
 'Leo']

- Converting all the numbers into the sheet name that the index of the number corresponds to

In [25]:
if inputted_sheet_names_list:
    for index, sheet in enumerate(inputted_sheet_names_list):
        # Checking if the sheet name was a number
        if isinstance(sheet, int) and str(sheet).isdigit():
            inputted_sheet_names_list[index] =  raw_data_sheet_names[sheet]

In [26]:
inputted_sheet_names_list

['Master List',
 'Jocelyn',
 'Aidan',
 'Albert',
 'Erika',
 'Ezra',
 'Meghan',
 'Naeliz',
 'Leo']

- Checking to see if all the sheets are in the excel file

In [27]:
if not set(inputted_sheet_names_list).issubset(raw_data_sheet_names):
    # Getting all the sheets that were not in the original spreadsheet
    not_included_sheet_names = set(inputted_sheet_names_list) - set(raw_data_sheet_names)
    raise ValueError("All the listed sheet names are not in {}".format(not_included_sheet_names))

## Finding the row for the header

- Headers are the row in a spreadsheet that has all the column names. Sometimes spreadsheets don't use the first row as the row with the column names. So, the row for the header will be asked or assumed to be the first row

# NOTE: If there is a set row that has the column names, then enter the row number in the quotation marks

In [28]:
# Making the row number a string, so that "0" is treated as a True value
all_header_row = "0"
if not all_header_row:
    all_header_row = False
else:
    all_header_row = int(all_header_row)


In [29]:
sheet_name_to_everything = defaultdict(dict)
for sheet in inputted_sheet_names_list:
    if all_header_row is False:

        print("\nCurrent Sheet Name: {}".format(sheet))    
        per_sheet_dataframe = pd.read_excel(raw_data_file_path, sheet_name=sheet, header=0)

        # Showing the columns that are chosen with the header being the 0th row
        print("Columns Names: {}".format(per_sheet_dataframe.columns))
        # Show the dataframe that would be created with the header being the 0th row
        print("First few rows of this dataframe:")
        print(pd.read_excel(raw_data_file_path, sheet_name=sheet, header=0).head())

        # Allowing the user the choose the row number for the header
        header_row = input("""Type the row number to be used as the header
        (AKA the row with the column name that you want to use.)
        If you want to keep the column names that were displayed, type 0.
        If you want to use a different row, then type the corresponding number. 

        The rows displayed in this cell are dataframes created from Pandas. 
        To use the row with the 0 index for column names, type 1. 
        For the row with the 1 index, it will be 2 and so on. i.e. 2

        If you are looking at the original spread sheet, remember that Python is zero indexed. 
        So the first row will be 0, second will be 1, and so on. 
        i.e. 1

        NOTE: If left blank, the original row that was used will be used.
        """).strip()

        if header_row == "":
            header_row = 0
        else:
            header_row = int(header_row)
    else:
        header_row = all_header_row
    
    # Checking if any of the column names are from empty cells
    column_names = "".join([str(col) for col in pd.read_excel(raw_data_file_path, sheet_name=sheet, header=header_row).columns])
    # If a column name came from an empty cell, it would have "Unnamed" in it
    if "Unnamed" in column_names:
        raise ValueError("""Not all the cells in the chosen row are filled in.
                         Please choose a row that has the name of the columns
                         
                         The values in this row are: {}""".format(column_names))
    else:
        sheet_name_to_everything[sheet]["header_row"] = header_row

In [30]:
sheet_name_to_everything

defaultdict(dict,
            {'Master List': {'header_row': 0},
             'Jocelyn': {'header_row': 0},
             'Aidan': {'header_row': 0},
             'Albert': {'header_row': 0},
             'Erika': {'header_row': 0},
             'Ezra': {'header_row': 0},
             'Meghan': {'header_row': 0},
             'Naeliz': {'header_row': 0},
             'Leo': {'header_row': 0}})

## Reading in all the spreadsheets

In [31]:
# Going through each sheet and creating a dataframe of it
for key, value in sheet_name_to_everything.items():
    value["original_behavior_recording_dataframe"] = pd.read_excel(raw_data_file_path, sheet_name=key, header=value["header_row"])

In [32]:
value["original_behavior_recording_dataframe"].head()

,Date,Cage,Box,Match,Scorer,Trial 1 Winner,Trial 2 Winner,Trial 3 Winner,Trial 4 Winner,Trial 5 Winner,...,Trial 13 Winner,Trial 14 Winner,Trial 15 Winner,Trial 16 Winner,Trial 17 Winner,Trial 18 Winner,Trial 19 Winner,Mouse 1 Wins,Mouse 2 Wins,Ties
0,2022-10-03,6,4,6.3 vs 6.4,Leo,6.3,6.4,6.4,6.4,6.3,...,6.3,6.4,6.3,6.4,6.3,6.3,6.3,12,7,0
1,2022-10-03,1,3,1.2 vs 1.4,Leo,1.2,1.4,1.4,1.2,1.2,...,1.2,1.4,1.2,1.2,1.2,1.2,1.2,16,3,0
2,2022-10-03,6,2,6.1 vs 6.3,Leo,6.1,6.3,6.1,6.1,6.1,...,6.3,6.1,6.3,6.3,6.3,6.3,6.3,7,12,0
3,2022-10-03,2,4,2.1 vs 2.4,Leo,2.4,tie,2.1,2.4,2.4,...,2.1,2.4,2.1,2.1,2.1,2.1,2.1,10,7,2
4,2022-10-03,5,2,5.1 vs 5.4,Leo,5.4,tie,5.4,5.4,5.4,...,5.4,5.4,5.4,5.4,5.4,5.4,5.4,0,18,1


In [33]:
value["original_behavior_recording_dataframe"].tail()

,Date,Cage,Box,Match,Scorer,Trial 1 Winner,Trial 2 Winner,Trial 3 Winner,Trial 4 Winner,Trial 5 Winner,...,Trial 13 Winner,Trial 14 Winner,Trial 15 Winner,Trial 16 Winner,Trial 17 Winner,Trial 18 Winner,Trial 19 Winner,Mouse 1 Wins,Mouse 2 Wins,Ties
4,2022-10-03,5,2,5.1 vs 5.4,Leo,5.4,tie,5.4,5.4,5.4,...,5.4,5.4,5.4,5.4,5.4,5.4,5.4,0,18,1
5,2022-10-04,4,2,4.2vs4.3,Leo,4.2,4.3,4.3,4.3,4.3,...,4.3,4.2,4.2,4.2,4.2,4.2,4.2,9,10,0
6,2022-10-04,5,1,5.1vs5.3,Leo,5.3,tie,5.3,5.3,5.3,...,5.3,5.3,5.3,5.3,5.3,5.3,5.3,0,18,1
7,2022-10-04,2,3,2.1vs2.3,Leo,2.1,2.1,2.1,2.1,2.1,...,2.1,tie,2.1,2.1,2.1,2.1,tie,15,1,3
8,2022-10-04,3,4,3.1vs3.2,Leo,3.2,3.2,3.2,3.1,3.1,...,3.2,tie,3.2,3.2,3.2,3.2,3.1,4,13,2


## Standarizing the Columns

- Making all the column names lower case and removing any extra spaces in the beginning and at the end
    - One dictionary per sheet

In [34]:
for key, value in sheet_name_to_everything.items():
    # Creating a dictionary that maps the original column name to the standarized one
    column_name_to_standarized = defaultdict(dict)
    for col in value["original_behavior_recording_dataframe"]:
        # Making the column name lower case and removing the spaces
        column_name_to_standarized[col] = "_".join(str(col).lower().strip().split(" "))
    value["column_name_to_standarized"] = column_name_to_standarized

In [35]:
value["column_name_to_standarized"]

defaultdict(dict,
            {'Date': 'date',
             'Cage': 'cage',
             'Box': 'box',
             'Match': 'match',
             'Scorer': 'scorer',
             'Trial 1 Winner': 'trial_1_winner',
             'Trial 2 Winner': 'trial_2_winner',
             'Trial 3 Winner': 'trial_3_winner',
             'Trial 4 Winner': 'trial_4_winner',
             'Trial 5 Winner': 'trial_5_winner',
             'Trial 6 Winner': 'trial_6_winner',
             'Trial 7 Winner': 'trial_7_winner',
             'Trial 8 Winner': 'trial_8_winner',
             'Trial 9 Winner': 'trial_9_winner',
             'Trial 10 Winner': 'trial_10_winner',
             'Trial 11 Winner': 'trial_11_winner',
             'Trial 12 Winner': 'trial_12_winner',
             'Trial 13 Winner': 'trial_13_winner',
             'Trial 14 Winner': 'trial_14_winner',
             'Trial 15 Winner': 'trial_15_winner',
             'Trial 16 Winner': 'trial_16_winner',
             'Trial 17 Winner': 'tr

In [36]:
# Renaming all the columns to the lower case and space removed version
for key, value in sheet_name_to_everything.items():
    value["processed_behavior_recording_dataframe"] = value["original_behavior_recording_dataframe"].rename(columns=value["column_name_to_standarized"])
    value["processed_behavior_recording_dataframe"]["sheet_name"] = key

In [37]:
value["processed_behavior_recording_dataframe"].head()

,date,cage,box,match,scorer,trial_1_winner,trial_2_winner,trial_3_winner,trial_4_winner,trial_5_winner,...,trial_14_winner,trial_15_winner,trial_16_winner,trial_17_winner,trial_18_winner,trial_19_winner,mouse_1_wins,mouse_2_wins,ties,sheet_name
0,2022-10-03,6,4,6.3 vs 6.4,Leo,6.3,6.4,6.4,6.4,6.3,...,6.4,6.3,6.4,6.3,6.3,6.3,12,7,0,Leo
1,2022-10-03,1,3,1.2 vs 1.4,Leo,1.2,1.4,1.4,1.2,1.2,...,1.4,1.2,1.2,1.2,1.2,1.2,16,3,0,Leo
2,2022-10-03,6,2,6.1 vs 6.3,Leo,6.1,6.3,6.1,6.1,6.1,...,6.1,6.3,6.3,6.3,6.3,6.3,7,12,0,Leo
3,2022-10-03,2,4,2.1 vs 2.4,Leo,2.4,tie,2.1,2.4,2.4,...,2.4,2.1,2.1,2.1,2.1,2.1,10,7,2,Leo
4,2022-10-03,5,2,5.1 vs 5.4,Leo,5.4,tie,5.4,5.4,5.4,...,5.4,5.4,5.4,5.4,5.4,5.4,0,18,1,Leo


In [38]:
reward_competition_df = sheet_name_to_everything[key]["processed_behavior_recording_dataframe"]

In [39]:
reward_competition_df.columns

Index(['date', 'cage', 'box', 'match', 'scorer', 'trial_1_winner',
       'trial_2_winner', 'trial_3_winner', 'trial_4_winner', 'trial_5_winner',
       'trial_6_winner', 'trial_7_winner', 'trial_8_winner', 'trial_9_winner',
       'trial_10_winner', 'trial_11_winner', 'trial_12_winner',
       'trial_13_winner', 'trial_14_winner', 'trial_15_winner',
       'trial_16_winner', 'trial_17_winner', 'trial_18_winner',
       'trial_19_winner', 'mouse_1_wins', 'mouse_2_wins', 'ties',
       'sheet_name'],
      dtype='object')

- Removing unnecessary columns

In [40]:
reward_competition_df = reward_competition_df.drop([col for col in reward_competition_df.columns if "wins" in col or "ties" in col], axis=1, errors="ignore")

In [41]:
columns_to_keep = [col for col in reward_competition_df.columns if "time" not in col.lower().strip()]

In [42]:
columns_to_keep

['date',
 'cage',
 'box',
 'match',
 'scorer',
 'trial_1_winner',
 'trial_2_winner',
 'trial_3_winner',
 'trial_4_winner',
 'trial_5_winner',
 'trial_6_winner',
 'trial_7_winner',
 'trial_8_winner',
 'trial_9_winner',
 'trial_10_winner',
 'trial_11_winner',
 'trial_12_winner',
 'trial_13_winner',
 'trial_14_winner',
 'trial_15_winner',
 'trial_16_winner',
 'trial_17_winner',
 'trial_18_winner',
 'trial_19_winner',
 'sheet_name']

In [43]:
reward_competition_df = reward_competition_df[columns_to_keep].copy()

In [44]:
reward_competition_df.head()

,date,cage,box,match,scorer,trial_1_winner,trial_2_winner,trial_3_winner,trial_4_winner,trial_5_winner,...,trial_11_winner,trial_12_winner,trial_13_winner,trial_14_winner,trial_15_winner,trial_16_winner,trial_17_winner,trial_18_winner,trial_19_winner,sheet_name
0,2022-10-03,6,4,6.3 vs 6.4,Leo,6.3,6.4,6.4,6.4,6.3,...,6.3,6.3,6.3,6.4,6.3,6.4,6.3,6.3,6.3,Leo
1,2022-10-03,1,3,1.2 vs 1.4,Leo,1.2,1.4,1.4,1.2,1.2,...,1.2,1.2,1.2,1.4,1.2,1.2,1.2,1.2,1.2,Leo
2,2022-10-03,6,2,6.1 vs 6.3,Leo,6.1,6.3,6.1,6.1,6.1,...,6.3,6.3,6.3,6.1,6.3,6.3,6.3,6.3,6.3,Leo
3,2022-10-03,2,4,2.1 vs 2.4,Leo,2.4,tie,2.1,2.4,2.4,...,2.1,2.1,2.1,2.4,2.1,2.1,2.1,2.1,2.1,Leo
4,2022-10-03,5,2,5.1 vs 5.4,Leo,5.4,tie,5.4,5.4,5.4,...,5.4,5.4,5.4,5.4,5.4,5.4,5.4,5.4,5.4,Leo


In [45]:
reward_competition_df = reward_competition_df.dropna(subset=["trial_1_winner"])

## Add both IDs as a column

In [46]:
reward_competition_df["animal_ids"] = reward_competition_df["match"].apply(lambda x: tuple(sorted([all_ids.strip() for all_ids in re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x)])))

In [47]:
reward_competition_df.head()

,date,cage,box,match,scorer,trial_1_winner,trial_2_winner,trial_3_winner,trial_4_winner,trial_5_winner,...,trial_12_winner,trial_13_winner,trial_14_winner,trial_15_winner,trial_16_winner,trial_17_winner,trial_18_winner,trial_19_winner,sheet_name,animal_ids
0,2022-10-03,6,4,6.3 vs 6.4,Leo,6.3,6.4,6.4,6.4,6.3,...,6.3,6.3,6.4,6.3,6.4,6.3,6.3,6.3,Leo,"(6.3, 6.4)"
1,2022-10-03,1,3,1.2 vs 1.4,Leo,1.2,1.4,1.4,1.2,1.2,...,1.2,1.2,1.4,1.2,1.2,1.2,1.2,1.2,Leo,"(1.2, 1.4)"
2,2022-10-03,6,2,6.1 vs 6.3,Leo,6.1,6.3,6.1,6.1,6.1,...,6.3,6.3,6.1,6.3,6.3,6.3,6.3,6.3,Leo,"(6.1, 6.3)"
3,2022-10-03,2,4,2.1 vs 2.4,Leo,2.4,tie,2.1,2.4,2.4,...,2.1,2.1,2.4,2.1,2.1,2.1,2.1,2.1,Leo,"(2.1, 2.4)"
4,2022-10-03,5,2,5.1 vs 5.4,Leo,5.4,tie,5.4,5.4,5.4,...,5.4,5.4,5.4,5.4,5.4,5.4,5.4,5.4,Leo,"(5.1, 5.4)"


In [48]:
reward_competition_df["cohort"] = cohort_name

- Adding strain information

# NOTE: If there are strains that are associated to each cage, then create a dictionary of cage numbers to strains inside the `{}`
- i.e. `cage_to_strain = {"1": "C57", "2": "C57", "3": "C57", "4": "CD1", "5": "CD1", "6": "CD1"}`

In [49]:
cage_to_strain = {}

In [50]:
reward_competition_df["strain"] = reward_competition_df["cage"].astype(str).map(cage_to_strain)

In [51]:
all_cages = "_".join([str(cage) for cage in sorted(reward_competition_df["cage"].unique())])

In [52]:
all_cages

'1_2_3_4_5_6'

In [53]:
earliest_date = reward_competition_df["date"].min()
latest_date = reward_competition_df["date"].max()

In [54]:
earliest_date = str(earliest_date.date()).replace("-", "")

In [55]:
latest_date = str(latest_date.date()).replace("-", "")

In [56]:
earliest_date

'20221003'

In [57]:
latest_date

'20221004'

- Creating a subfolder to put the Elo rating Spreadsheets

In [58]:
elo_rating_spreadsheet_output_directory = os.path.join(".", "proc", "elo_rating_spread_sheets", "{}".format(protocol_name))

In [59]:
elo_rating_spreadsheet_output_directory

'./proc/elo_rating_spread_sheets/reward_competition'

In [60]:
os.makedirs(elo_rating_spreadsheet_output_directory, exist_ok=True)

In [61]:
file_name = "{}_{}_trial_as_columns_cages_{}_date_{}_{}.csv".format(cohort_name, protocol_name, all_cages, earliest_date, latest_date)

reward_competition_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name))

In [62]:
reward_competition_df = reward_competition_df.drop(columns=["sheet_name", "scorer"], errors="ignore")

## Melt 

In [63]:
reward_competition_df["index"] = reward_competition_df.index
reward_competition_df = reward_competition_df.reset_index(drop=True)

In [64]:
reward_competition_df.head()

,date,cage,box,match,trial_1_winner,trial_2_winner,trial_3_winner,trial_4_winner,trial_5_winner,trial_6_winner,...,trial_14_winner,trial_15_winner,trial_16_winner,trial_17_winner,trial_18_winner,trial_19_winner,animal_ids,cohort,strain,index
0,2022-10-03,6,4,6.3 vs 6.4,6.3,6.4,6.4,6.4,6.3,6.4,...,6.4,6.3,6.4,6.3,6.3,6.3,"(6.3, 6.4)",,NaN,0
1,2022-10-03,1,3,1.2 vs 1.4,1.2,1.4,1.4,1.2,1.2,1.2,...,1.4,1.2,1.2,1.2,1.2,1.2,"(1.2, 1.4)",,NaN,1
2,2022-10-03,6,2,6.1 vs 6.3,6.1,6.3,6.1,6.1,6.1,6.1,...,6.1,6.3,6.3,6.3,6.3,6.3,"(6.1, 6.3)",,NaN,2
3,2022-10-03,2,4,2.1 vs 2.4,2.4,tie,2.1,2.4,2.4,2.4,...,2.4,2.1,2.1,2.1,2.1,2.1,"(2.1, 2.4)",,NaN,3
4,2022-10-03,5,2,5.1 vs 5.4,5.4,tie,5.4,5.4,5.4,5.4,...,5.4,5.4,5.4,5.4,5.4,5.4,"(5.1, 5.4)",,NaN,4


In [65]:
melted_reward_competition_df = reward_competition_df.melt(id_vars=["index", "date", "cage", "box", "match", "animal_ids"], 
        var_name="trial", 
        value_name="winner")

In [66]:
melted_reward_competition_df.head()

,index,date,cage,box,match,animal_ids,trial,winner
0,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_1_winner,6.3
1,1,2022-10-03,1,3,1.2 vs 1.4,"(1.2, 1.4)",trial_1_winner,1.2
2,2,2022-10-03,6,2,6.1 vs 6.3,"(6.1, 6.3)",trial_1_winner,6.1
3,3,2022-10-03,2,4,2.1 vs 2.4,"(2.1, 2.4)",trial_1_winner,2.4
4,4,2022-10-03,5,2,5.1 vs 5.4,"(5.1, 5.4)",trial_1_winner,5.4


In [67]:
melted_reward_competition_df.tail()

,index,date,cage,box,match,animal_ids,trial,winner
184,4,2022-10-03,5,2,5.1 vs 5.4,"(5.1, 5.4)",strain,NaN
185,5,2022-10-04,4,2,4.2vs4.3,"(4.2, 4.3)",strain,NaN
186,6,2022-10-04,5,1,5.1vs5.3,"(5.1, 5.3)",strain,NaN
187,7,2022-10-04,2,3,2.1vs2.3,"(2.1, 2.3)",strain,NaN
188,8,2022-10-04,3,4,3.1vs3.2,"(3.1, 3.2)",strain,NaN


- Dropping all rows that don't contain Winner

In [68]:
melted_reward_competition_df = melted_reward_competition_df.dropna(subset="winner")

In [69]:
melted_reward_competition_df["winner"].unique()

array([6.3, 1.2, 6.1, 2.4, 5.4, 4.2, 5.3, 2.1, 3.2, 6.4, 1.4, 'tie', 4.3,
       3.1, 2.3, ''], dtype=object)

In [70]:
melted_reward_competition_df["keep_row"] = melted_reward_competition_df["winner"].apply(lambda x: True if "tie" in str(x).lower() or re.match(r'^-?\d+(?:\.\d+)$', str(x)) else False)

In [71]:
melted_reward_competition_df = melted_reward_competition_df[melted_reward_competition_df["keep_row"]]

In [72]:
melted_reward_competition_df["winner"].unique()

array([6.3, 1.2, 6.1, 2.4, 5.4, 4.2, 5.3, 2.1, 3.2, 6.4, 1.4, 'tie', 4.3,
       3.1, 2.3], dtype=object)

- Making all the ids into string

In [73]:
melted_reward_competition_df["winner"] = melted_reward_competition_df["winner"].astype(str)
melted_reward_competition_df["winner"] = melted_reward_competition_df["winner"].apply(lambda x: x.lower().strip())

In [74]:
melted_reward_competition_df["animal_ids"].unique()

array([('6.3', '6.4'), ('1.2', '1.4'), ('6.1', '6.3'), ('2.1', '2.4'),
       ('5.1', '5.4'), ('4.2', '4.3'), ('5.1', '5.3'), ('2.1', '2.3'),
       ('3.1', '3.2')], dtype=object)

- Making a different column for ties

In [75]:
melted_reward_competition_df["match_is_tie"] = melted_reward_competition_df["winner"].apply(lambda x: True if "tie" in x.lower().strip() else False)

- Replacing tie with the first animal id for elo score plotting

In [76]:
melted_reward_competition_df["winner"] = melted_reward_competition_df.apply(lambda x: x["animal_ids"][0] if x["match_is_tie"] else x["winner"], axis=1)

In [77]:
melted_reward_competition_df[melted_reward_competition_df["match_is_tie"]]

,index,date,cage,box,match,animal_ids,trial,winner,keep_row,match_is_tie
12,3,2022-10-03,2,4,2.1 vs 2.4,"(2.1, 2.4)",trial_2_winner,2.1,True,True
13,4,2022-10-03,5,2,5.1 vs 5.4,"(5.1, 5.4)",trial_2_winner,5.1,True,True
15,6,2022-10-04,5,1,5.1vs5.3,"(5.1, 5.3)",trial_2_winner,5.1,True,True
75,3,2022-10-03,2,4,2.1 vs 2.4,"(2.1, 2.4)",trial_9_winner,2.1,True,True
88,7,2022-10-04,2,3,2.1vs2.3,"(2.1, 2.3)",trial_10_winner,2.1,True,True
89,8,2022-10-04,3,4,3.1vs3.2,"(3.1, 3.2)",trial_10_winner,3.1,True,True
124,7,2022-10-04,2,3,2.1vs2.3,"(2.1, 2.3)",trial_14_winner,2.1,True,True
125,8,2022-10-04,3,4,3.1vs3.2,"(3.1, 3.2)",trial_14_winner,3.1,True,True
169,7,2022-10-04,2,3,2.1vs2.3,"(2.1, 2.3)",trial_19_winner,2.1,True,True


- Dropping all rows that don't have trial in the name for the trial section

In [78]:
melted_reward_competition_df = melted_reward_competition_df[melted_reward_competition_df["trial"].str.contains('trial')]

- Getting the number of the trial so that we can order by number(instead of string, which would make 11 come before 2)

In [79]:
melted_reward_competition_df["trial_number"] = melted_reward_competition_df["trial"].apply(lambda x: int(x.lower().strip("trial").strip("winner").strip("_")))

In [80]:
melted_reward_competition_df = melted_reward_competition_df.sort_values(["index", "trial_number"]).reset_index(drop=True)

In [81]:
melted_reward_competition_df.head()

,index,date,cage,box,match,animal_ids,trial,winner,keep_row,match_is_tie,trial_number
0,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_1_winner,6.3,True,False,1
1,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_2_winner,6.4,True,False,2
2,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_3_winner,6.4,True,False,3
3,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_4_winner,6.4,True,False,4
4,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_5_winner,6.3,True,False,5


In [82]:
melted_reward_competition_df.head(n=100)

,index,date,cage,box,match,animal_ids,trial,winner,keep_row,match_is_tie,trial_number
0,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_1_winner,6.3,True,False,1
1,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_2_winner,6.4,True,False,2
2,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_3_winner,6.4,True,False,3
3,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_4_winner,6.4,True,False,4
4,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_5_winner,6.3,True,False,5
...,...,...,...,...,...,...,...,...,...,...,...
95,5,2022-10-04,4,2,4.2vs4.3,"(4.2, 4.3)",trial_1_winner,4.2,True,False,1
96,5,2022-10-04,4,2,4.2vs4.3,"(4.2, 4.3)",trial_2_winner,4.3,True,False,2
97,5,2022-10-04,4,2,4.2vs4.3,"(4.2, 4.3)",trial_3_winner,4.3,True,False,3
98,5,2022-10-04,4,2,4.2vs4.3,"(4.2, 4.3)",trial_4_winner,4.3,True,False,4


In [83]:
melted_reward_competition_df["trial_number"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])

## Add a column for the loser ID

In [84]:
melted_reward_competition_df.columns

Index(['index', 'date', 'cage', 'box', 'match', 'animal_ids', 'trial',
       'winner', 'keep_row', 'match_is_tie', 'trial_number'],
      dtype='object')

In [85]:
melted_reward_competition_df.head()

,index,date,cage,box,match,animal_ids,trial,winner,keep_row,match_is_tie,trial_number
0,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_1_winner,6.3,True,False,1
1,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_2_winner,6.4,True,False,2
2,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_3_winner,6.4,True,False,3
3,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_4_winner,6.4,True,False,4
4,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_5_winner,6.3,True,False,5


In [86]:
melted_reward_competition_df["loser"] = melted_reward_competition_df.apply(lambda x: (list(set(x["animal_ids"]) - set([x["winner"]]))[0]), axis=1)

In [87]:
melted_reward_competition_df["loser"].unique()

array(['6.4', '6.3', '1.4', '1.2', '6.1', '2.1', '2.4', '5.1', '5.4',
       '4.3', '4.2', '5.3', '2.3', '3.1', '3.2'], dtype=object)

In [88]:
melted_reward_competition_df.head(n=5)

,index,date,cage,box,match,animal_ids,trial,winner,keep_row,match_is_tie,trial_number,loser
0,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_1_winner,6.3,True,False,1,6.4
1,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_2_winner,6.4,True,False,2,6.3
2,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_3_winner,6.4,True,False,3,6.3
3,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_4_winner,6.4,True,False,4,6.3
4,0,2022-10-03,6,4,6.3 vs 6.4,"(6.3, 6.4)",trial_5_winner,6.3,True,False,5,6.4


In [89]:
melted_reward_competition_df.tail(n=5)

,index,date,cage,box,match,animal_ids,trial,winner,keep_row,match_is_tie,trial_number,loser
166,8,2022-10-04,3,4,3.1vs3.2,"(3.1, 3.2)",trial_15_winner,3.2,True,False,15,3.1
167,8,2022-10-04,3,4,3.1vs3.2,"(3.1, 3.2)",trial_16_winner,3.2,True,False,16,3.1
168,8,2022-10-04,3,4,3.1vs3.2,"(3.1, 3.2)",trial_17_winner,3.2,True,False,17,3.1
169,8,2022-10-04,3,4,3.1vs3.2,"(3.1, 3.2)",trial_18_winner,3.2,True,False,18,3.1
170,8,2022-10-04,3,4,3.1vs3.2,"(3.1, 3.2)",trial_19_winner,3.1,True,False,19,3.2


## Adding the session number

- We are adding the session number to all the trials. The session number is counting the number of recording sessions that have happened up until that trial. Usually, each session in the spreadsheet is divided up by a session's first row having the date filled in. So we will label a new session when a date is filled in.

# Getting the Session number differences

- Getting the indexes of where each new session starts. So that we can add the session number to each row.

In [90]:
melted_reward_competition_df["session_number_difference"] = melted_reward_competition_df["date"].astype('category').cat.codes.diff()


## Calculating Elo rating

- Example calculation

In [91]:
calculation.calculate_elo_rating(subject_elo_rating=1000, agent_elo_rating=2000, score=1)

1019.9

In [92]:
calculation.update_elo_rating(winner_id="A", loser_id="B")

defaultdict(<function elorating.calculation.update_elo_rating.<locals>.<lambda>()>,
            {'A': 1010.0, 'B': 990.0})

## Get the Elo rating for all the events

- Going through each row or interaction and calculating the new Elo rating for the winner and loser. This will create a new dataframe based off of the processed behavioral recording dataframe

In [93]:
cage_to_elo_rating_dict = defaultdict(dict)
for cage in melted_reward_competition_df["cage"].unique():
    cage_df = melted_reward_competition_df[melted_reward_competition_df["cage"] == cage]
    cage_to_elo_rating_dict[cage] = calculation.iterate_elo_rating_calculation_for_dataframe(dataframe=cage_df, winner_id_column="winner", loser_id_column="loser", additional_columns=melted_reward_competition_df.columns, tie_column="match_is_tie")

In [94]:
cage_to_elo_rating_dict.keys()

dict_keys([6, 1, 2, 5, 4, 3])

In [95]:
cage_to_elo_rating_dict[list(cage_to_elo_rating_dict.keys())[0]][0]

{'total_match_number': 1,
 'subject_id': '6.3',
 'agent_id': '6.4',
 'original_elo_rating': 1000,
 'updated_elo_rating': 1010.0,
 'win_draw_loss': 1,
 'subject_ranking': 1,
 'agent_ranking': 2,
 'pairing_index': 0,
 'index': 0,
 'date': Timestamp('2022-10-03 00:00:00'),
 'cage': 6,
 'box': 4,
 'match': '6.3 vs 6.4',
 'animal_ids': ('6.3', '6.4'),
 'trial': 'trial_1_winner',
 'winner': '6.3',
 'keep_row': True,
 'match_is_tie': False,
 'trial_number': 1,
 'loser': '6.4',
 'session_number_difference': nan}

- Turning the dictionary into a dataframe

In [96]:
all_cage_elo_rating_list = []
for key in cage_to_elo_rating_dict.keys():
    cage_elo_rating_df = pd.DataFrame.from_dict(cage_to_elo_rating_dict[key], orient="index")
    cage_elo_rating_df.insert(0, 'total_trial_number', range(0, 0 + len(cage_elo_rating_df)))
    print(cage_elo_rating_df)
    all_cage_elo_rating_list.append(cage_elo_rating_df)


    total_trial_number  total_match_number subject_id agent_id  \
0                    0                   1        6.3      6.4   
1                    1                   1        6.4      6.3   
2                    2                   2        6.4      6.3   
3                    3                   2        6.3      6.4   
4                    4                   3        6.4      6.3   
..                 ...                 ...        ...      ...   
71                  71                  36        6.1      6.3   
72                  72                  37        6.3      6.1   
73                  73                  37        6.1      6.3   
74                  74                  38        6.3      6.1   
75                  75                  38        6.1      6.3   

    original_elo_rating  updated_elo_rating  win_draw_loss  subject_ranking  \
0                1000.0              1010.0              1                1   
1                1000.0               990.0      

In [97]:
all_cage_elo_rating_df = pd.concat(all_cage_elo_rating_list)

In [98]:
all_cage_elo_rating_df.head()

,total_trial_number,total_match_number,subject_id,agent_id,original_elo_rating,updated_elo_rating,win_draw_loss,subject_ranking,agent_ranking,pairing_index,...,box,match,animal_ids,trial,winner,keep_row,match_is_tie,trial_number,loser,session_number_difference
0,0,1,6.3,6.4,1000.0,1010.0,1.0,1,2,0,...,4,6.3 vs 6.4,"(6.3, 6.4)",trial_1_winner,6.3,True,False,1,6.4,NaN
1,1,1,6.4,6.3,1000.0,990.0,0.0,2,1,1,...,4,6.3 vs 6.4,"(6.3, 6.4)",trial_1_winner,6.3,True,False,1,6.4,NaN
2,2,2,6.4,6.3,990.0,1000.6,1.0,1,2,0,...,4,6.3 vs 6.4,"(6.3, 6.4)",trial_2_winner,6.4,True,False,2,6.3,0.0
3,3,2,6.3,6.4,1010.0,999.4,0.0,2,1,1,...,4,6.3 vs 6.4,"(6.3, 6.4)",trial_2_winner,6.4,True,False,2,6.3,0.0
4,4,3,6.4,6.3,1000.6,1010.6,1.0,1,2,0,...,4,6.3 vs 6.4,"(6.3, 6.4)",trial_3_winner,6.4,True,False,3,6.3,0.0


In [99]:
all_cage_elo_rating_df.tail()

,total_trial_number,total_match_number,subject_id,agent_id,original_elo_rating,updated_elo_rating,win_draw_loss,subject_ranking,agent_ranking,pairing_index,...,box,match,animal_ids,trial,winner,keep_row,match_is_tie,trial_number,loser,session_number_difference
33,33,17,3.1,3.2,940.4,933.7,0.0,2,1,1,...,4,3.1vs3.2,"(3.1, 3.2)",trial_17_winner,3.2,True,False,17,3.1,0.0
34,34,18,3.2,3.1,1066.3,1072.7,1.0,1,2,0,...,4,3.1vs3.2,"(3.1, 3.2)",trial_18_winner,3.2,True,False,18,3.1,0.0
35,35,18,3.1,3.2,933.7,927.3,0.0,2,1,1,...,4,3.1vs3.2,"(3.1, 3.2)",trial_18_winner,3.2,True,False,18,3.1,0.0
36,36,19,3.1,3.2,927.3,941.3,1.0,2,1,0,...,4,3.1vs3.2,"(3.1, 3.2)",trial_19_winner,3.1,True,False,19,3.2,0.0
37,37,19,3.2,3.1,1072.7,1058.7,0.0,1,2,1,...,4,3.1vs3.2,"(3.1, 3.2)",trial_19_winner,3.1,True,False,19,3.2,0.0


In [100]:
all_cage_elo_rating_df[all_cage_elo_rating_df["match_is_tie"]]

,total_trial_number,total_match_number,subject_id,agent_id,original_elo_rating,updated_elo_rating,win_draw_loss,subject_ranking,agent_ranking,pairing_index,...,box,match,animal_ids,trial,winner,keep_row,match_is_tie,trial_number,loser,session_number_difference
2,2,2,2.1,2.4,990.0,990.6,0.5,2,1,0,...,4,2.1 vs 2.4,"(2.1, 2.4)",trial_2_winner,2.1,True,True,2,2.4,0.0
3,3,2,2.4,2.1,1010.0,1009.4,0.5,1,2,1,...,4,2.1 vs 2.4,"(2.1, 2.4)",trial_2_winner,2.1,True,True,2,2.4,0.0
16,16,9,2.1,2.4,956.2,958.7,0.5,2,1,0,...,4,2.1 vs 2.4,"(2.1, 2.4)",trial_9_winner,2.1,True,True,9,2.4,0.0
17,17,9,2.4,2.1,1043.8,1041.3,0.5,1,2,1,...,4,2.1 vs 2.4,"(2.1, 2.4)",trial_9_winner,2.1,True,True,9,2.4,0.0
56,56,29,2.1,2.3,1085.6,1082.0,0.5,1,3,0,...,3,2.1vs2.3,"(2.1, 2.3)",trial_10_winner,2.1,True,True,10,2.3,0.0
57,57,29,2.3,2.1,954.6,958.2,0.5,3,1,1,...,3,2.1vs2.3,"(2.1, 2.3)",trial_10_winner,2.1,True,True,10,2.3,0.0
64,64,33,2.1,2.3,1100.7,1096.4,0.5,1,3,0,...,3,2.1vs2.3,"(2.1, 2.3)",trial_14_winner,2.1,True,True,14,2.3,0.0
65,65,33,2.3,2.1,939.5,943.8,0.5,3,1,1,...,3,2.1vs2.3,"(2.1, 2.3)",trial_14_winner,2.1,True,True,14,2.3,0.0
74,74,38,2.1,2.3,1118.3,1113.2,0.5,1,3,0,...,3,2.1vs2.3,"(2.1, 2.3)",trial_19_winner,2.1,True,True,19,2.3,0.0
75,75,38,2.3,2.1,921.9,927.0,0.5,3,1,1,...,3,2.1vs2.3,"(2.1, 2.3)",trial_19_winner,2.1,True,True,19,2.3,0.0


In [101]:
if cage_to_strain:
    all_cage_elo_rating_df["strain"] = all_cage_elo_rating_df["cage"].astype(str).map(cage_to_strain)

- Adding the cohort

In [102]:
all_cage_elo_rating_df["experiment_type"] = protocol_name
all_cage_elo_rating_df["cohort"] = cohort_name

In [103]:
all_cage_elo_rating_df.head()

,total_trial_number,total_match_number,subject_id,agent_id,original_elo_rating,updated_elo_rating,win_draw_loss,subject_ranking,agent_ranking,pairing_index,...,animal_ids,trial,winner,keep_row,match_is_tie,trial_number,loser,session_number_difference,experiment_type,cohort
0,0,1,6.3,6.4,1000.0,1010.0,1.0,1,2,0,...,"(6.3, 6.4)",trial_1_winner,6.3,True,False,1,6.4,NaN,reward_competition,
1,1,1,6.4,6.3,1000.0,990.0,0.0,2,1,1,...,"(6.3, 6.4)",trial_1_winner,6.3,True,False,1,6.4,NaN,reward_competition,
2,2,2,6.4,6.3,990.0,1000.6,1.0,1,2,0,...,"(6.3, 6.4)",trial_2_winner,6.4,True,False,2,6.3,0.0,reward_competition,
3,3,2,6.3,6.4,1010.0,999.4,0.0,2,1,1,...,"(6.3, 6.4)",trial_2_winner,6.4,True,False,2,6.3,0.0,reward_competition,
4,4,3,6.4,6.3,1000.6,1010.6,1.0,1,2,0,...,"(6.3, 6.4)",trial_3_winner,6.4,True,False,3,6.3,0.0,reward_competition,


In [104]:
all_cage_elo_rating_df.head()

,total_trial_number,total_match_number,subject_id,agent_id,original_elo_rating,updated_elo_rating,win_draw_loss,subject_ranking,agent_ranking,pairing_index,...,animal_ids,trial,winner,keep_row,match_is_tie,trial_number,loser,session_number_difference,experiment_type,cohort
0,0,1,6.3,6.4,1000.0,1010.0,1.0,1,2,0,...,"(6.3, 6.4)",trial_1_winner,6.3,True,False,1,6.4,NaN,reward_competition,
1,1,1,6.4,6.3,1000.0,990.0,0.0,2,1,1,...,"(6.3, 6.4)",trial_1_winner,6.3,True,False,1,6.4,NaN,reward_competition,
2,2,2,6.4,6.3,990.0,1000.6,1.0,1,2,0,...,"(6.3, 6.4)",trial_2_winner,6.4,True,False,2,6.3,0.0,reward_competition,
3,3,2,6.3,6.4,1010.0,999.4,0.0,2,1,1,...,"(6.3, 6.4)",trial_2_winner,6.4,True,False,2,6.3,0.0,reward_competition,
4,4,3,6.4,6.3,1000.6,1010.6,1.0,1,2,0,...,"(6.3, 6.4)",trial_3_winner,6.4,True,False,3,6.3,0.0,reward_competition,


## Saving the Elo Score Dataframes

- Create folders to save processed data

- Adding the cage information for each cage

In [105]:
all_cage_elo_rating_df.head()

,total_trial_number,total_match_number,subject_id,agent_id,original_elo_rating,updated_elo_rating,win_draw_loss,subject_ranking,agent_ranking,pairing_index,...,animal_ids,trial,winner,keep_row,match_is_tie,trial_number,loser,session_number_difference,experiment_type,cohort
0,0,1,6.3,6.4,1000.0,1010.0,1.0,1,2,0,...,"(6.3, 6.4)",trial_1_winner,6.3,True,False,1,6.4,NaN,reward_competition,
1,1,1,6.4,6.3,1000.0,990.0,0.0,2,1,1,...,"(6.3, 6.4)",trial_1_winner,6.3,True,False,1,6.4,NaN,reward_competition,
2,2,2,6.4,6.3,990.0,1000.6,1.0,1,2,0,...,"(6.3, 6.4)",trial_2_winner,6.4,True,False,2,6.3,0.0,reward_competition,
3,3,2,6.3,6.4,1010.0,999.4,0.0,2,1,1,...,"(6.3, 6.4)",trial_2_winner,6.4,True,False,2,6.3,0.0,reward_competition,
4,4,3,6.4,6.3,1000.6,1010.6,1.0,1,2,0,...,"(6.3, 6.4)",trial_3_winner,6.4,True,False,3,6.3,0.0,reward_competition,


In [106]:
all_cage_elo_rating_df[all_cage_elo_rating_df["win_draw_loss"] == 0.5]

,total_trial_number,total_match_number,subject_id,agent_id,original_elo_rating,updated_elo_rating,win_draw_loss,subject_ranking,agent_ranking,pairing_index,...,animal_ids,trial,winner,keep_row,match_is_tie,trial_number,loser,session_number_difference,experiment_type,cohort
2,2,2,2.1,2.4,990.0,990.6,0.5,2,1,0,...,"(2.1, 2.4)",trial_2_winner,2.1,True,True,2,2.4,0.0,reward_competition,
3,3,2,2.4,2.1,1010.0,1009.4,0.5,1,2,1,...,"(2.1, 2.4)",trial_2_winner,2.1,True,True,2,2.4,0.0,reward_competition,
16,16,9,2.1,2.4,956.2,958.7,0.5,2,1,0,...,"(2.1, 2.4)",trial_9_winner,2.1,True,True,9,2.4,0.0,reward_competition,
17,17,9,2.4,2.1,1043.8,1041.3,0.5,1,2,1,...,"(2.1, 2.4)",trial_9_winner,2.1,True,True,9,2.4,0.0,reward_competition,
56,56,29,2.1,2.3,1085.6,1082.0,0.5,1,3,0,...,"(2.1, 2.3)",trial_10_winner,2.1,True,True,10,2.3,0.0,reward_competition,
57,57,29,2.3,2.1,954.6,958.2,0.5,3,1,1,...,"(2.1, 2.3)",trial_10_winner,2.1,True,True,10,2.3,0.0,reward_competition,
64,64,33,2.1,2.3,1100.7,1096.4,0.5,1,3,0,...,"(2.1, 2.3)",trial_14_winner,2.1,True,True,14,2.3,0.0,reward_competition,
65,65,33,2.3,2.1,939.5,943.8,0.5,3,1,1,...,"(2.1, 2.3)",trial_14_winner,2.1,True,True,14,2.3,0.0,reward_competition,
74,74,38,2.1,2.3,1118.3,1113.2,0.5,1,3,0,...,"(2.1, 2.3)",trial_19_winner,2.1,True,True,19,2.3,0.0,reward_competition,
75,75,38,2.3,2.1,921.9,927.0,0.5,3,1,1,...,"(2.1, 2.3)",trial_19_winner,2.1,True,True,19,2.3,0.0,reward_competition,


In [107]:
id_to_final_elo_rating_dict = defaultdict(dict)
for index, subject_id in enumerate(sorted(all_cage_elo_rating_df["subject_id"].unique())):
    per_subject_df = all_cage_elo_rating_df[all_cage_elo_rating_df["subject_id"] == subject_id]
    id_to_final_elo_rating_dict[index]["subject_id"] = subject_id

    id_to_final_elo_rating_dict[index]["final_elo_rating"] = per_subject_df.iloc[-1]["updated_elo_rating"]
    id_to_final_elo_rating_dict[index]["cohort"] = per_subject_df.iloc[-1]["cohort"]
    id_to_final_elo_rating_dict[index]["strain"] = per_subject_df.iloc[-1]["strain"]
    id_to_final_elo_rating_dict[index]["cage"] = per_subject_df.iloc[-1]["cage"]

    

KeyError: 'strain'

In [ ]:
id_to_final_elo_rating_dict

In [ ]:
id_to_final_elo_rating_df = pd.DataFrame.from_dict(id_to_final_elo_rating_dict, orient="index")
# Adding protocol name
id_to_final_elo_rating_df["experiment_type"] = protocol_name
# Adding rank
id_to_final_elo_rating_df["rank"] = id_to_final_elo_rating_df.groupby("cage")["final_elo_rating"].rank("dense", ascending=False)
# Sorting by cage and then id
id_to_final_elo_rating_df = id_to_final_elo_rating_df.sort_values(by=['cage', "subject_id"], ascending=True).reset_index(drop=True)

In [ ]:
id_to_final_elo_rating_df["rank"] = id_to_final_elo_rating_df.groupby("cage")["final_elo_rating"].rank("dense", ascending=False)

In [ ]:
id_to_final_elo_rating_df = id_to_final_elo_rating_df.sort_values(by=['cage', "subject_id"], ascending=True).reset_index(drop=True)

In [ ]:
id_to_final_elo_rating_df.head()

In [ ]:
id_to_final_elo_rating_df.tail()

- Getting the cage numbersm

In [ ]:
all_cages_list = []
# Creating a list of all the cage numbers
for key, value in sheet_name_to_everything.items():
    try:
        for cage in reward_competition_df["cage"].unique():
            all_cages_list.append(str(cage))
    except:
        print("WARNING: {} does not have cage number as columns".format(key))
        warnings.warn("Look at warning from above or below")

In [ ]:
try:
    all_cages_string = "-".join(sorted([sheet.lower().strip("cage").strip() for sheet in all_cages_list]))
    all_cages_string = "cages-{}".format(all_cages_string)
    print("String of cage names to use for file name: {}".format(all_cages_string))
except: 
    warnings.warn("WARNING: There are no cage numbers to make a title out of")
    all_cages_string = None

# Plotting the Elo Score by match number

In [ ]:
plot_output_directory = "./proc/plots/reward_competition/cage_{}_date_{}_{}".format(all_cages, earliest_date, latest_date)

In [ ]:
plot_output_directory

In [ ]:
os.makedirs(plot_output_directory, exist_ok=True)

In [ ]:
all_cage_elo_rating_df["subject_id"] = all_cage_elo_rating_df["subject_id"].astype(str)
all_cage_elo_rating_df["agent_id"] = all_cage_elo_rating_df["agent_id"].astype(str)

In [ ]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (18,10)

In [ ]:
for cage in all_cage_elo_rating_df["cage"].unique():
    fig, ax = plt.subplots()
    plt.rcParams["figure.figsize"] = (18,10)
    per_cage_df = all_cage_elo_rating_df[all_cage_elo_rating_df["cage"] == cage]
       
    for index in per_cage_df["index"].unique():
        first_session_in_trial = per_cage_df[per_cage_df["index"] == index].iloc[0]["total_trial_number"]
        plt.vlines(x=[first_session_in_trial - 0.5], ymin=700, ymax=1300, colors='black', linestyle='dashed')

    
    
    # Drawing a line for each subject
    for subject in sorted(per_cage_df["subject_id"].unique()):
        # Getting all the rows with the current subject
        subject_df = per_cage_df[per_cage_df["subject_id"] == subject]
        # Making the dates into days after the first session by subtracting all the dates by the first date
        plt.plot(subject_df["total_trial_number"], subject_df["updated_elo_rating"], '-o', label=subject)
    
    # Labeling the X/Y Axis and the title
    ax.set_xlabel("Trial Number")
    ax.set_ylabel("Elo Score")
    ax.set_title("{} Elo Score for {}: Cage {}".format(" ".join(cohort_name.split("_")).capitalize(), string.capwords(" ".join(protocol_name.split("_"))), cage))
    # To show the legend
    ax.legend(loc="upper left")
    plt.xticks(rotation=90)
    plt.ylim(700, 1300) 
    file_name = "{}_{}_elo_rating_{}_date_{}_{}.png".format(cohort_name, protocol_name, cage, earliest_date, latest_date)
    plt.savefig(os.path.join(plot_output_directory, file_name))  
    
    plt.show()

# Saving the Dataframes

In [ ]:
elo_rating_spreadsheet_output_directory = os.path.join(".", "proc", "elo_rating_spread_sheets", "{}".format(protocol_name))

In [ ]:
elo_rating_spreadsheet_output_directory

In [ ]:
os.makedirs(elo_rating_spreadsheet_output_directory, exist_ok=True)

In [ ]:
file_name_parts_separated = [cohort_name, all_cages_string, prefix_name, earliest_date, latest_date]
file_name_parts_combined = "_".join([part for part in file_name_parts_separated if part])

file_name_full = "{}_elo-rating-history.csv".format(file_name_parts_combined)
print(file_name_full)
all_cage_elo_rating_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name_full))

In [ ]:
file_name_full = "{}_final-elo-rating.csv".format(file_name_parts_combined)
print(file_name_full)
id_to_final_elo_rating_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name_full))

## Seeing which subject is the dominant or submissive

- Grouping all the rows with the same pair

In [ ]:
all_processed_behavior_recording_df = melted_reward_competition_df

In [ ]:
all_processed_behavior_recording_df.head()

In [ ]:
all_processed_behavior_recording_df.columns

In [ ]:
['date', 'cage', 'match', 'animal_ids', 'trial', 'winner', 'tie', 'loser']

- Removing columns that would be unnecessary for the pairings

In [ ]:
all_processed_behavior_recording_df = all_processed_behavior_recording_df[['date', 'cage', 'match', 'animal_ids', 'trial', 'winner', 'match_is_tie', 'loser']]
all_processed_behavior_recording_df['processed_cage_number'] = all_processed_behavior_recording_df['cage'].astype(int)

In [ ]:
all_processed_behavior_recording_df.head()

- Adding a tie to the list of winners and losers if it's a tie

In [ ]:
all_processed_behavior_recording_df["winner"] = all_processed_behavior_recording_df.apply(lambda x: "tie" if x["match_is_tie"] else x["winner"], axis=1)
all_processed_behavior_recording_df["loser"] = all_processed_behavior_recording_df.apply(lambda x: "tie" if x["match_is_tie"] else x["loser"], axis=1)

In [ ]:
all_processed_behavior_recording_df[all_processed_behavior_recording_df["match_is_tie"]].head()

- Making a list of all the wins and loses

In [ ]:
all_wins_per_pair = all_processed_behavior_recording_df.groupby(["animal_ids", "date"])['winner'].apply(list)
all_loses_per_pair = all_processed_behavior_recording_df.groupby(["animal_ids", "date"])['loser'].apply(list)

In [ ]:
all_competition_per_pair_df = pd.concat([all_wins_per_pair, all_loses_per_pair], axis=1).reset_index()

In [ ]:
all_competition_per_pair_df = all_competition_per_pair_df.rename(columns={k: "rc_" + k for k in all_competition_per_pair_df.columns})

In [ ]:
all_competition_per_pair_df

- Seeing how often the winners or losers change

In [ ]:
# Dropping all the ties
all_competition_per_pair_df["rc_winner_no_ties"] = all_competition_per_pair_df.apply(lambda row: [x for x in row["rc_winner"] if x in row["rc_animal_ids"]], axis=1)
all_competition_per_pair_df["rc_loser_no_ties"] = all_competition_per_pair_df.apply(lambda row: [x for x in row["rc_loser"] if x in row["rc_animal_ids"]], axis=1)

In [ ]:
# Comparing the each item with the item after it to see if they are the same of not
all_competition_per_pair_df["rc_different_result_as_previous"] = all_competition_per_pair_df["rc_winner_no_ties"].apply(lambda x: [True if left != right else False for (left, right) in zip(x[1:], x[:-1])])

In [ ]:
all_competition_per_pair_df["rc_number_of_switches"] = all_competition_per_pair_df["rc_different_result_as_previous"].apply(lambda x: sum(x))

In [ ]:
all_competition_per_pair_df

# NOTE: This assumes that Ties aren't the most common outcome for a given pair

## Aggregate all the wins/loses across reward competition sessions

- Getting the average number of switches between winner and loser

In [ ]:
# Renaming column to not be confused with previous version
average_switches_per_pair_df = all_competition_per_pair_df.groupby('rc_animal_ids').mean().rename({'rc_number_of_switches': 'rc_average_number_of_switches'}, axis=1)  # new method

- Combining all the trials into one list

In [ ]:
all_sessions_per_pair_df = pd.concat([all_competition_per_pair_df.groupby('rc_animal_ids')['rc_winner'].apply(list), all_competition_per_pair_df.groupby('rc_animal_ids')['rc_loser'].apply(list)], axis=1)
all_sessions_per_pair_df = pd.concat([all_sessions_per_pair_df, average_switches_per_pair_df], axis=1)
all_sessions_per_pair_df = all_sessions_per_pair_df.reset_index()

In [ ]:
all_sessions_per_pair_df.head()

- Flattening the list

In [ ]:
all_sessions_per_pair_df["rc_winner"] = all_sessions_per_pair_df["rc_winner"].apply(lambda x: [element for innerList in x for element in innerList])
all_sessions_per_pair_df["rc_loser"] = all_sessions_per_pair_df["rc_loser"].apply(lambda x: [element for innerList in x for element in innerList])

In [ ]:
all_sessions_per_pair_df.head()

- Calculating overall winner and loser based on who has the most wins/loses

In [ ]:
# Dropping all the ties
all_sessions_per_pair_df["rc_winner_no_ties"] = all_sessions_per_pair_df.apply(lambda row: [x for x in row["rc_winner"] if x in row["rc_animal_ids"]], axis=1)
all_sessions_per_pair_df["rc_loser_no_ties"] = all_sessions_per_pair_df.apply(lambda row: [x for x in row["rc_loser"] if x in row["rc_animal_ids"]], axis=1)

In [ ]:
# Putting the first ID for the winner and second for the loser if there is equal number of wins and loses 
all_sessions_per_pair_df["rc_averaged_winner"] = all_sessions_per_pair_df.apply(lambda x: Counter(x["rc_winner_no_ties"]).most_common(1)[0][0] if x["rc_winner_no_ties"].count(Counter(x["rc_winner_no_ties"]).most_common(1)[0][1]) != len(x["rc_winner_no_ties"]) / 2 else x["rc_animal_ids"][0], axis=1)
all_sessions_per_pair_df["rc_averaged_loser"] = all_sessions_per_pair_df.apply(lambda x: Counter(x["rc_loser_no_ties"]).most_common(1)[0][0] if x["rc_loser_no_ties"].count(Counter(x["rc_loser_no_ties"]).most_common(1)[0][1]) != len(x["rc_winner_no_ties"]) / 2 else x["rc_animal_ids"][1], axis=1)


- Checking to see if any of the IDs are the same or are ties

In [ ]:
all_sessions_per_pair_df["rc_averaged_loser"].unique()

In [ ]:
all_sessions_per_pair_df["rc_averaged_winner"].unique()

In [ ]:
all_sessions_per_pair_df[all_sessions_per_pair_df["rc_averaged_loser"] == all_sessions_per_pair_df["rc_averaged_winner"]]

- Counting the number of wins and loses

In [ ]:
all_sessions_per_pair_df["rc_winner_count"] = all_sessions_per_pair_df.apply(lambda x: x["rc_winner"].count(x["rc_averaged_winner"]), axis=1)
all_sessions_per_pair_df["rc_loser_count"] = all_sessions_per_pair_df.apply(lambda x: x["rc_winner"].count(x["rc_averaged_loser"]), axis=1)
all_sessions_per_pair_df["rc_tie_count"] = all_sessions_per_pair_df.apply(lambda x: x["rc_winner"].count("tie"), axis=1)
all_sessions_per_pair_df["rc_all_match_count_including_ties"] = all_sessions_per_pair_df["rc_winner"].apply(lambda x: len(x))

- Comparing the number of wins vs loses

In [ ]:
all_sessions_per_pair_df["rc_winner_count_minus_loser_count"] = all_sessions_per_pair_df["rc_winner_count"] - all_sessions_per_pair_df["rc_loser_count"]
# winner number / (winner + loser number with no ties)
all_sessions_per_pair_df["rc_win_to_win_plus_lost_ratio"] = all_sessions_per_pair_df.apply(lambda x: x["rc_winner_count"] / len(x["rc_winner_no_ties"]), axis=1)
# winner number / (winner + loser + tie)
all_sessions_per_pair_df["rc_win_to_all_ratio"] = all_sessions_per_pair_df.apply(lambda x: x["rc_winner_count"] / x["rc_all_match_count_including_ties"], axis=1)
# all_sessions_per_pair_df["rc_is_win_to_all_ratio_tie"] = all_sessions_per_pair_df["rc_win_to_all_ratio"].apply(lambda x: True if x < 0.5 else False)
all_sessions_per_pair_df["rc_is_win_to_win_and_loss_ratio_tie"] = all_sessions_per_pair_df["rc_win_to_win_plus_lost_ratio"].apply(lambda x: True if x < 0.6 else False)

In [ ]:
all_sessions_per_pair_df["rc_is_win_to_win_and_loss_ratio_tie"]

- Comparing the number of ties to the number of matches

In [ ]:
all_sessions_per_pair_df["rc_tie_to_all_ratio"] = all_sessions_per_pair_df.apply(lambda x: x["rc_tie_count"] / x["rc_all_match_count_including_ties"], axis=1)


In [ ]:
all_sessions_per_pair_df[~all_sessions_per_pair_df["rc_is_win_to_win_and_loss_ratio_tie"]]

In [ ]:
all_sessions_per_pair_df.head()

- Checking to see if there are any pairs with more than two sessions

In [ ]:
all_sessions_per_pair_df[all_sessions_per_pair_df["rc_all_match_count_including_ties"] > 40]

- Adding the cage information

In [ ]:
dropped_cuplicate_all_processed_behavior_recording_df = all_processed_behavior_recording_df[["animal_ids", "processed_cage_number"]].drop_duplicates()

In [ ]:
dropped_cuplicate_all_processed_behavior_recording_df

In [ ]:
pair_to_cage = pd.Series(dropped_cuplicate_all_processed_behavior_recording_df["processed_cage_number"].values, index=dropped_cuplicate_all_processed_behavior_recording_df["animal_ids"]).to_dict()

In [ ]:
pair_to_cage

In [ ]:
all_sessions_per_pair_df["processed_cage_number"] = all_sessions_per_pair_df["rc_animal_ids"].map(pair_to_cage)

In [ ]:
all_sessions_per_pair_df

In [ ]:
all_sessions_per_pair_df["cohort"] = cohort_name
all_sessions_per_pair_df["processed_cage_number"] = all_sessions_per_pair_df["processed_cage_number"].astype(int).astype(str)

In [ ]:
all_sessions_per_pair_df.head()

- Verifying if all the stats are correct

In [ ]:
all_sessions_per_pair_df["rc_winner"][0].count(all_sessions_per_pair_df["rc_animal_ids"][0][0])

In [ ]:
all_sessions_per_pair_df["rc_winner"][0].count(all_sessions_per_pair_df["rc_animal_ids"][1][0])

In [ ]:
all_sessions_per_pair_df["rc_winner"][0].count("tie")

In [ ]:
len(all_sessions_per_pair_df["rc_winner"][0])

In [ ]:
len(all_sessions_per_pair_df["rc_winner_no_ties"][0])

In [ ]:
all_sessions_per_pair_df.tail()

# Getting the number of wins per winner and loser

In [ ]:
all_competition_per_pair_df["cohort"] = cohort_name

In [ ]:
all_competition_per_pair_df["processed_cage_number"] = all_competition_per_pair_df["rc_animal_ids"].map(pair_to_cage).astype(str)

In [ ]:
all_competition_per_pair_df["strain"] = all_competition_per_pair_df["processed_cage_number"].map(cage_to_strain)

In [ ]:
all_competition_per_pair_df.head()

- Calculating overall winner and loser based on who has the most wins/loses

In [ ]:
# Dropping all the ties
all_competition_per_pair_df["rc_winner_no_ties"] = all_competition_per_pair_df.apply(lambda row: [x for x in row["rc_winner"] if x in row["rc_animal_ids"]], axis=1)
all_competition_per_pair_df["rc_loser_no_ties"] = all_competition_per_pair_df.apply(lambda row: [x for x in row["rc_loser"] if x in row["rc_animal_ids"]], axis=1)

In [ ]:
# Putting the first ID for the winner and second for the loser if there is equal number of wins and loses 
all_competition_per_pair_df["rc_averaged_winner"] = all_competition_per_pair_df.apply(lambda x: Counter(x["rc_winner_no_ties"]).most_common(1)[0][0] if x["rc_winner_no_ties"].count(Counter(x["rc_winner_no_ties"]).most_common(1)[0][1]) != len(x["rc_winner_no_ties"]) / 2 else x["rc_animal_ids"][0], axis=1)
all_competition_per_pair_df["rc_averaged_loser"] = all_competition_per_pair_df.apply(lambda x: Counter(x["rc_loser_no_ties"]).most_common(1)[0][0] if x["rc_loser_no_ties"].count(Counter(x["rc_loser_no_ties"]).most_common(1)[0][1]) != len(x["rc_winner_no_ties"]) / 2 else x["rc_animal_ids"][1], axis=1)


In [ ]:
Counter(all_competition_per_pair_df["rc_winner_no_ties"][0]).most_common(1)[0][1]

- Checking to see if any of the IDs are the same or are ties

In [ ]:
all_competition_per_pair_df["rc_averaged_loser"].unique()

In [ ]:
all_competition_per_pair_df["rc_averaged_winner"].unique()

- Counting the number of wins and loses

In [ ]:
all_competition_per_pair_df["rc_winner_count"] = all_competition_per_pair_df.apply(lambda x: x["rc_winner"].count(x["rc_averaged_winner"]), axis=1)
all_competition_per_pair_df["rc_loser_count"] = all_competition_per_pair_df.apply(lambda x: x["rc_winner"].count(x["rc_averaged_loser"]), axis=1)
all_competition_per_pair_df["rc_tie_count"] = all_competition_per_pair_df.apply(lambda x: x["rc_winner"].count("tie"), axis=1)
all_competition_per_pair_df["rc_all_match_count_including_ties"] = all_competition_per_pair_df["rc_winner"].apply(lambda x: len(x))

In [ ]:
all_competition_per_pair_df.head()

- Comparing the number of wins vs loses

In [ ]:
all_competition_per_pair_df["rc_winner_count_minus_loser_count"] = all_competition_per_pair_df["rc_winner_count"] - all_competition_per_pair_df["rc_loser_count"]
# winner number / (winner + loser number with no ties)
all_competition_per_pair_df["rc_win_to_win_plus_lost_ratio"] = all_competition_per_pair_df.apply(lambda x: x["rc_winner_count"] / len(x["rc_winner_no_ties"]), axis=1)
# winner number / (winner + loser + tie)
all_competition_per_pair_df["rc_win_to_all_ratio"] = all_competition_per_pair_df.apply(lambda x: x["rc_winner_count"] / x["rc_all_match_count_including_ties"], axis=1)
all_competition_per_pair_df["rc_is_win_to_win_and_loss_ratio_tie"] = all_competition_per_pair_df["rc_win_to_win_plus_lost_ratio"].apply(lambda x: True if x < 0.6 else False)

In [ ]:
all_competition_per_pair_df["rc_is_win_to_win_and_loss_ratio_tie"]

- Comparing the number of ties to the number of matches

In [ ]:
all_competition_per_pair_df["rc_tie_to_all_ratio"] = all_competition_per_pair_df.apply(lambda x: x["rc_tie_count"] / x["rc_all_match_count_including_ties"], axis=1)


In [ ]:
all_competition_per_pair_df.head()

- Saving the competition dataframe

In [ ]:
file_name = "{}_{}_grouped_by_pairs_cage_{}_date_{}_{}.csv".format(cohort_name, protocol_name, all_cages, earliest_date, latest_date)


In [ ]:
elo_rating_spreadsheet_output_directory

In [ ]:
file_name

In [ ]:
file_name = "{}_{}_all_competition_cage_{}_date_{}_{}.csv".format(cohort_name, protocol_name, all_cages, earliest_date, latest_date)


In [ ]:
elo_rating_spreadsheet_output_directory

In [ ]:
file_name

In [ ]:
all_competition_per_pair_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name))

# Saving the spreadsheets

In [ ]:
file_name = "{}_{}_grouped_by_pairs_cage_{}_date_{}_{}.csv".format(cohort_name, protocol_name, all_cages, earliest_date, latest_date)


In [ ]:
elo_rating_spreadsheet_output_directory

In [ ]:
file_name

In [ ]:
all_sessions_per_pair_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name))